In [ ]:
%pip install pandas==2.2.*

In [ ]:
import requests
import os
import re
import json
import pandas as pd


def get_lcsc_numbers(csv_path: str = "assessments.csv") -> list[int]:
    df = pd.read_csv(csv_path)
    return df["lcsc"].dropna().astype(int).tolist()  # List of LCSCs


def extract_assessment(content: str):
    try:
        json_str = re.search(r"\{.*\}", content, re.DOTALL).group(0)
        data = json.loads(json_str)
        return {
            "pinswap_rating": int(data.get("pinswap_rating", 0)),
            "overall_rating": int(data.get("overall_rating", 0)),
            "reasoning": data.get("reasoning", "").strip(),
        }
    except Exception as e:
        print(f"Failed to parse assessment: {e}")
        return {"pinswap_rating": 0, "overall_rating": 0, "reasoning": "Assessment parsing failed"}


def LLM_assess_for_pinswaps(symbol: str, api_key: str):
    try:
        prompt = f"""Analyze this KiCad symbol for pin swap errors and provide ratings:
        Return JSON format only: {{"pinswap_rating": 0-100, "overall_rating": 0-100, "reasoning": "..."}}
        Symbol:
        ```
        {symbol}
        ```"""

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            json={
                "model": "deepseek/deepseek-r1:free",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.05,
            },
            timeout=15,
        ).json()

        return extract_assessment(response["choices"][0]["message"]["content"])
    except Exception as e:
        print(f"Assessment failed: {e}")
        print(response)
        return {"pinswap_rating": 0, "overall_rating": 0, "reasoning": "Assessment request failed"}


def process_symbol_file(input_path: str, output_path: str, api_key: str) -> None:
    # results = {}
    lcsc_numbers = get_lcsc_numbers(output_path)

    try:
        with open(input_path, "r", encoding="utf-8") as f:
            current_symbol = []
            lcsc = None
            name = None

            for line in f:
                if '(generator "CDFER"' in line:
                    print(f"Skipping autogen file: {input_path}")
                    return

                if '(symbol "' in line and "_" not in line or line == ")\n":
                    if name and lcsc and current_symbol:
                        if lcsc not in lcsc_numbers:
                            symbol_code = "".join(current_symbol)
                            results = {}
                            results[lcsc] = LLM_assess_for_pinswaps(symbol_code, api_key)
                            if results[lcsc]["reasoning"] != "Assessment parsing failed":
                                results[lcsc]["name"] = name
                                results[lcsc]["path"] = input_path
                                save_assessments(results, output_path)
                        current_symbol.clear()

                    if '(symbol "' in line:
                        name = re.search(r'"([^"]+)"', line).group(1)
                        print(f"Processing: {name}")

                if '(property "LCSC"' in line:
                    lcsc = int(re.search(r"C(\d+)", line).group(1))

                current_symbol.append(line)

    except Exception as e:
        print(f"File processing failed for {input_path}: {e}")


def process_symbol_directory(dir_path: str, output_path: str, api_key: str) -> None:
    for filename in os.listdir(dir_path):
        if filename.endswith(".kicad_sym"):
            full_path = os.path.join(dir_path, filename)
            process_symbol_file(full_path, output_path, api_key)

    return


# Reporting interface
def save_assessments(results, csv_path: str = "assessments.csv") -> None:
    """Save results to CSV with pandas, appending data and removing duplicates."""
    # Convert results to DataFrame
    formatted_data = [
        {
            "lcsc": lcsc,
            "Component": data["name"],
            "Path": data["path"],
            "SwapsRating": data["pinswap_rating"],
            "TotalRating": data["overall_rating"],
            "Comments": data["reasoning"].replace("\n", " ").replace('"', ""),
        }
        for lcsc, data in results.items()
    ]

    new_df = pd.DataFrame(formatted_data)

    # Append to existing data or create new file
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        # Keep last occurrence of each component
        combined_df = combined_df.drop_duplicates(subset=["Component"], keep="last")
    else:
        combined_df = new_df

    combined_df = combined_df.sort_values(by="SwapsRating")

    # Save consolidated data
    combined_df.to_csv(csv_path, index=False)
    return


# Usage examples
if __name__ == "__main__":
    API_KEY = "Put-Your-OpenRouter-Key-Here"

    # Test single file
    # process_symbol_file("symbols/JLCPCB-Memory.kicad_sym", "pinswap.csv", API_KEY)

    # Full directory scan
    process_symbol_directory("symbols", "pinswap.csv", API_KEY)

Processing: Comparator, LM393DR2G
Processing: Digital Potentiometer, SPI, MCP4131-104E/SN
Processing: Op-Amp, LM2904DR2G
Processing: Op-Amp, LM324DT
Processing: Op-Amp, LM358DR2G
Processing: Op-Amp, LMV321IDBVR
Processing: Op-Amp, MCP6002T-I/SN
Processing: Op-Amp, NE5532DR
Processing: Op-Amp, OP07CDR
Processing: Op-Amp, TL072CDT
Processing: Switch, CD4051BM96
Processing: Switch, CD4052BM96
Skipping autogen file: symbols\JLCPCB-Capacitors.kicad_sym
Processing: Battery Holder, CR2032
Processing: Headphone Jack, 3.5mm
Processing: Screw Terminal, 4Px5.08mm, 20A
Processing: Spring Terminal, 2Px12mm, 15A
Processing: Tactile Button, 160gf
Processing: Crystal, 12MHz, 20pF
Processing: Crystal, 16MHz, 9pF
Processing: Crystal, 25MHz, 11pF
Processing: Crystal, 32.768kHz, 13pF
Processing: Crystal, 8MHz, 20pF, ±10ppm
Processing: Crystal, 8MHz, 20pF, ±20ppm
Processing: Bridge Rectifier, DB107S
Processing: Bridge Rectifier, MB10S
Processing: Package, BAT54TW
Processing: Package, H5VU25U
Processing: Pa